In [1]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
print(tf.__version__)
from tensorflow.contrib import rnn
import random
import collections
import time

TensorFlow 1.x selected.
1.15.2


In [2]:
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

In [3]:
start_time = time.time()
def elapsed(sec):
  if sec<60:
      return str(sec) + " sec"
  elif sec<(60*60):
      return str(sec/60) + " min"
  else:
    return (str(sec/(60*60)) + " hr")

In [4]:
logs_path = '/content/rnn_words'
writer = tf.summary.FileWriter(logs_path)

training_file = 'belling_the_cat.txt'


In [5]:
def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [word for i in range(len(content)) for word in content[i].split()]
    content = np.array(content)
    return content  

In [6]:
training_data = read_data(training_file)
training_data

array(['long', 'ago', ',', 'the', 'mice', 'had', 'a', 'general',
       'council', 'to', 'consider', 'what', 'measures', 'they', 'could',
       'take', 'to', 'outwit', 'their', 'common', 'enemy', ',', 'the',
       'cat', '.', 'some', 'said', 'this', ',', 'and', 'some', 'said',
       'that', 'but', 'at', 'last', 'a', 'young', 'mouse', 'got', 'up',
       'and', 'said', 'he', 'had', 'a', 'proposal', 'to', 'make', ',',
       'which', 'he', 'thought', 'would', 'meet', 'the', 'case', '.',
       'you', 'will', 'all', 'agree', ',', 'said', 'he', ',', 'that',
       'our', 'chief', 'danger', 'consists', 'in', 'the', 'sly', 'and',
       'treacherous', 'manner', 'in', 'which', 'the', 'enemy',
       'approaches', 'us', '.', 'now', ',', 'if', 'we', 'could',
       'receive', 'some', 'signal', 'of', 'her', 'approach', ',', 'we',
       'could', 'easily', 'escape', 'from', 'her', '.', 'i', 'venture',
       ',', 'therefore', ',', 'to', 'propose', 'that', 'a', 'small',
       'bell', 'be', 'pr

In [7]:
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary


In [8]:
dictionary, reverse_dictionary = build_dataset(training_data)
dictionary['cat']

9

In [9]:
vocab_size = len(dictionary)

# Learning Rate-> v=u+at 
# EPOCHS-> best LR? 
# LR_final = 0, LR_init = 0.001, t= EPOCH 
# degradation_rate = - (LR_final - LR_initial / EPOCHS ) 

# Parameters
learning_rate = 0.001
training_iters = 50
display_step = 10
n_input = 3

# number of units in RNN cell
n_hidden = 512


In [10]:
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# TF-> PLACEHOLDERS (input, output), VARIABLES (weights, bias) 
# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

In [12]:
def RNN(x, weights, biases):

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])
    x = tf.split(x,n_input,1)

    # 2-layer LSTM, each layer has n_hidden units.
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

   # rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [13]:
# predictions will be compared against the Y (known output)
pred = RNN(x, weights, biases) 
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [15]:

with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)
    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    print("Run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    #print("Point your web browser to: http://localhost:6006/")
    while True:
        prompt = "%s words: " % n_input
        sentence = input(prompt)
        sentence = sentence.strip()
        words = sentence.split(' ')
        if len(words) != n_input:
            continue
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            for i in range(32):
                keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
            print(sentence)
        except:
            print("Word not in dictionary")

Iter= 10, Average Loss= 5.959339, Average Accuracy= 0.00%
['mouse', 'got', 'up'] - [and] vs [young]
Iter= 20, Average Loss= 8.556083, Average Accuracy= 0.00%
['which', 'the', 'enemy'] - [approaches] vs [in]
Iter= 30, Average Loss= 9.170168, Average Accuracy= 0.00%
['attached', 'by', 'a'] - [ribbon] vs [and]
Iter= 40, Average Loss= 11.680335, Average Accuracy= 0.00%
[',', 'until', 'an'] - [old] vs [applause]
Iter= 50, Average Loss= 8.481762, Average Accuracy= 0.00%
['easy', 'to', 'propose'] - [impossible] vs [is]
Optimization Finished!
Elapsed time:  28.15463238954544 min
Run on command line.
	tensorboard --logdir=/content/rnn_words
3 words: the long mouse
the long mouse impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible impossible im

KeyboardInterrupt: ignored